음성합성을 위한 데이터셋

In [ ]:
!pip install -q librosa soundfile

import warnings
warnings.filterwarnings("ignore")

import os
import time
import csv
import librosa
import soundfile as sf
from base64 import b64decode
from google.colab.output import eval_js
from IPython.display import display, HTML, Audio

# ─── 1. Tacotron 학습용 대본 리스트 ────────────────────────────────────────────
scripts = [
    "안녕하세요, 오늘 날씨는 맑고 화창합니다.",
    "서울에서 부산까지 KTX로 두 시간이 걸립니다.",
    "집 앞 공원에서 아이들이 공놀이를 하고 있습니다.",
    "다음 주 금요일 오후 세 시에 회의가 잡혀 있습니다.",
    "한국의 전통 음식 중 하나인 비빔밥은 맛과 영양이 뛰어납니다.",
    "컴퓨터 비전과 자연어 처리는 인공지능의 주요 분야입니다.",
    "오늘 아침에 마신 커피 한 잔이 정신을 맑게 해 주었습니다.",
    "자동차를 운전할 때는 항상 안전벨트를 착용해야 합니다.",
    "프로그래밍 언어 파이썬은 배우기 쉽고 강력합니다.",
    "바닷가에서 들려오는 파도 소리는 마음을 편안하게 해 줍니다."
]

# ─── 2. 녹음 파일 저장 디렉터리 및 메타 리스트 ───────────────────────────────
os.makedirs("recordings", exist_ok=True)
metadata = []  # (wav_path, transcript) 튜플을 저장할 리스트

# ─── 3. 녹음 및 WAV 변환 함수 ─────────────────────────────────────────────────
def record_script(text, filename, duration=7):
    # 화면에 대본 표시
    display(HTML(f"<h3>다음 문장을 따라 읽어주세요:</h3>"
                 f"<p style='font-size:20px;'>{text}</p>"))

    # JS로 녹음 및 카운트다운 UI 실행 (async IIFE로 감싸기)
    js = f"""
(async () => {{
  const duration = {duration};
  const timerDiv = document.createElement('div');
  timerDiv.id = 'countdown';
  timerDiv.style.fontSize = '20px';
  timerDiv.style.marginBottom = '10px';
  document.body.appendChild(timerDiv);

  const stream = await navigator.mediaDevices.getUserMedia({{ audio: true }});
  const recorder = new MediaRecorder(stream);
  const chunks = [];
  recorder.ondataavailable = e => chunks.push(e.data);
  recorder.start();

  for (let i = duration; i >= 0; i--) {{
    timerDiv.innerText = `녹음 중: ${{i}}초 남음`;
    await new Promise(r => setTimeout(r, 1000));
  }}

  recorder.stop();
  await new Promise(r => recorder.onstop = r);
  timerDiv.innerText = '녹음 완료!';

  const blob = new Blob(chunks);
  const reader = new FileReader();
  reader.readAsDataURL(blob);
  await new Promise(r => reader.onloadend = r);
  timerDiv.remove();
  return reader.result.split(',')[1];
}})();
    """
    b64data = eval_js(js)
    audio_bytes = b64decode(b64data)

    # WEBM 파일로 임시 저장
    path_webm = os.path.join("recordings", filename + ".webm")
    with open(path_webm, "wb") as f:
        f.write(audio_bytes)

    # librosa로 로드 후 soundfile로 WAV 저장 (16-bit PCM)
    audio, sr = librosa.load(path_webm, sr=441000) #sampling rate 44.1kHz으로 설정
    path_wav = os.path.join("recordings", filename + ".wav")
    sf.write(path_wav, audio, sr, subtype='PCM_16')

    # 임시 WEBM 파일 삭제
    os.remove(path_webm)

    print(f"저장 완료 (WAV): {path_wav}")
    # 녹음된 WAV 재생
    display(Audio(path_wav, autoplay=False))
    return path_wav

# ─── 4. 대본별 순차 녹음 진행 ──────────────────────────────────────────────────
for idx, text in enumerate(scripts, start=1):
    fname = f"utt_{idx:02d}"
    wav_path = record_script(text, fname, duration=7)
    metadata.append((wav_path, text))
    time.sleep(1)

# ─── 5. CSV로 대본 메타데이터 저장 ─────────────────────────────────────────────
csv_path = os.path.join("recordings", "transcripts.csv")
with open(csv_path, "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["filename", "transcript"])
    for wav_path, transcript in metadata:
        writer.writerow([os.path.basename(wav_path), transcript])

print(f"\n모든 녹음 완료! 대본 CSV 저장: {csv_path}")

ERROR: Operation cancelled by user


In [ ]:
# /content/recordings 디렉터리 생성 (기존 삭제 후)
!rm -rf /content/recordings
!mkdir -p /content/recordings

# recordings.zip 내용 /content/recordings 에 풀기
!unzip -o recordings.zip -d /content/recordings

# 결과 확인
!ls /content/recordings

VITS 기반 음성합성 학습 및 생성

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import json # JSON 파일 수정을 위해 추가

# --- 1. VITS GitHub 저장소 클론 및 작업 디렉토리 설정 ---
print("--- 1. VITS GitHub 저장소 클론 및 작업 디렉토리 설정 ---")
vits_repo_path = "/content/vits"
if not os.path.exists(vits_repo_path):
    print(f"Cloning VITS repository to {vits_repo_path}...")
    !git clone https://github.com/jaywalnut310/vits.git {vits_repo_path}
else:
    print(f"VITS repository already exists at {vits_repo_path}.")

# VITS 디렉토리로 이동 (모든 작업의 기준)
%cd {vits_repo_path}
print(f"Current working directory: {os.getcwd()}")


# --- 2. 필수 라이브러리 설치 (★requirements.txt 문제 우회를 위해 수정됨★) ---
print("\n--- 2. 필수 라이브러리 설치 ---")
try:
    print("Installing common dependencies manually to bypass requirements.txt issues...")
    # PyTorch 설치 (GPU 사용 시, Colab 기본 CUDA 11.8)
    # CPU 사용 시: !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
    !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

    # 파일 목록 생성 스크립트에서 사용:
    !pip install pandas scikit-learn

    # 오디오 처리 및 유틸리티 (numpy, matplotlib 버전 고정 제거하여 최신 호환 버전 설치 시도)
    !pip install librosa soundfile matplotlib scipy numpy einops tensorboardX

    # 텍스트 전처리 (한국어 cleaners에 필요):
    !pip install text_normalization phonemizer Unidecode # Unidecode 패키지가 unidecode 모듈을 포함합니다.

    print("Dependency installation completed successfully.")

except Exception as e:
    print(f"Error during dependency installation: {e}")
    print("Please check the error and ensure all required packages are installed.")
    # 중요한 오류이므로 여기서 스크립트 실행을 중단할 수 있도록 raise
    raise RuntimeError("Failed to install all necessary dependencies.")


# --- 3. 코랩 런타임 재시작 (★새로운 라이브러리 적용을 위해 필수★) ---
# 이 셀에서 런타임이 재시작되므로, 이후의 모든 코드는 재시작 후에 새로 실행됩니다.
# 따라서 이 셀을 실행한 후에는 다시 스크립트의 맨 위부터 실행해야 합니다.
print("\n--- 3. 런타임 재시작 중... (필수) ---")
print("이 메시지 다음에 런타임이 재시작됩니다. 재시작 후 스크립트를 처음부터 다시 실행해주세요.")
os.kill(os.getpid(), 9) # 런타임 강제 종료 (코랩 재시작과 동일 효과)


# --- 4. 데이터셋 파일 목록 생성 (런타임 재시작 후 이어서 실행) ---
# (이 부분은 런타임 재시작 후 다시 실행됩니다)
print("\n--- 4. 데이터셋 파일 목록 생성 ---")
transcripts_file_path = '/content/recordings/transcripts.csv' # 사용자 데이터셋 transcripts.csv 경로
audio_root_path = '/content/recordings/' # 사용자 데이터셋 .wav 파일들이 있는 폴더

output_filelist_dir = './filelists'
os.makedirs(output_filelist_dir, exist_ok=True)

if os.path.exists(transcripts_file_path):
    print(f"Reading transcripts from {transcripts_file_path}...")
    df = pd.read_csv(transcripts_file_path, sep=',', header=0)
    df['filename'] = df['filename'].apply(lambda x: os.path.join(audio_root_path, x))

    train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

    train_filelist_path = os.path.join(output_filelist_dir, 'train_filelist.txt')
    val_filelist_path = os.path.join(output_filelist_dir, 'val_filelist.txt')

    train_df[['filename', 'transcript']].to_csv(train_filelist_path, sep='|', index=False, header=False, encoding='utf-8')
    val_df[['filename', 'transcript']].to_csv(val_filelist_path, sep='|', index=False, header=False, encoding='utf-8')

    print(f"Train file list created: {train_filelist_path}")
    print(f"Validation file list created: {val_filelist_path}")

    # 파일 내용 일부 확인
    print("\n--- train_filelist.txt sample content ---")
    with open(train_filelist_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            print(line.strip())
            if i >= 2: break
else:
    print(f"Error: transcripts.csv not found at {transcripts_file_path}. Please upload your dataset.")
    raise FileNotFoundError(f"transcripts.csv not found at {transcripts_file_path}")


# --- 5. VITS 설정 파일 준비 및 수정 (런타임 재시작 후 이어서 실행) ---
print("\n--- 5. VITS 설정 파일 준비 및 수정 ---")
config_source_path = 'configs/ljs_base.json' # 기본 설정 파일
config_target_path = 'configs/my_korean_vits.json' # 사용자 정의 설정 파일

if os.path.exists(config_source_path):
    !cp {config_source_path} {config_target_path}
    print(f"Copied {config_source_path} to {config_target_path}")

    # JSON 파일 로드 및 수정
    try:
        with open(config_target_path, 'r', encoding='utf-8') as f:
            config = json.load(f)

        # 데이터 관련 설정 수정
        config['data']['training_files'] = "filelists/train_filelist.txt"
        config['data']['validation_files'] = "filelists/val_filelist.txt"
        config['data']['text_cleaners'] = ["korean_cleaners"]
        config['data']['sampling_rate'] = 22050 # 사용자 WAV 파일의 실제 샘플링 레이트에 맞춰야 함!
        config['data']['n_speakers'] = 0 # 단일 화자

        # 모델 관련 설정 수정 (단일 화자이므로 gin_channels 0)
        config['model']['gin_channels'] = 0

        # 학습 관련 설정 (필요시 batch_size, epochs, fp16_run 등 조절)
        config['train']['batch_size'] = 16 # Colab GPU 메모리에 맞춰 조절
        config['train']['fp16_run'] = True # GPU 사용 시 True, CPU 사용 시 False

        with open(config_target_path, 'w', encoding='utf-8') as f:
            json.dump(config, f, indent=2, ensure_ascii=False) # 한글 깨짐 방지를 위해 ensure_ascii=False
        print(f"Configuration file {config_target_path} updated successfully.")

    except Exception as e:
        print(f"Error updating JSON configuration: {e}")
        print("Please check the JSON file manually for correctness.")
else:
    print(f"Error: Source config file {config_source_path} not found. Cannot create custom config.")


# --- 6. (선택 사항) TensorBoard 실행 (런타임 재시작 후 이어서 실행) ---
print("\n--- 6. (Optional) TensorBoard 실행 ---")
try:
    %load_ext tensorboard
    %tensorboard --logdir logs/
    print("TensorBoard started. Check the link above for monitoring.")
except Exception as e:
    print(f"Could not start TensorBoard: {e}. It might be a Colab environment issue.")


# --- 7. VITS 학습 실행 (런타임 재시작 후 이어서 실행) ---
print("\n--- 7. VITS 학습 실행 ---")
model_name = "my_korean_model" # 학습 로그 및 체크포인트가 저장될 이름
config_file = config_target_path # 사용할 설정 파일

print(f"Starting VITS training with config: {config_file}, model name: {model_name}")
!python train.py -c {config_file} -m {model_name}

print("\n--- VITS 학습 스크립트 실행 완료 ---")

FastSpeech2음성합성 과정

In [ ]:
!pip install -q librosa soundfile g2pk matplotlib

In [ ]:
# 녹음 전체 코드: 이미 사용자께서 완성한 부분 (생략 가능)
# 결과물:
# /content/recordings/
# ├── utt_01.wav ~ utt_10.wav
# └── transcripts.csv

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

# 경로 설정
wav_path = "/content/recordings/recordings/utt_01.wav"

# 로드 및 mel 추출
# sampling rate를 44.1kHz로 하여 sampling한다. 그리고 fft 한뒤 mel filter bank를 80개로 하여 mel-spectrogram을 생성한다.
y, sr = librosa.load(wav_path, sr=44100)
mel = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=1024, hop_length=256, n_mels=80)
mel_db = librosa.power_to_db(mel, ref=np.max) #mel의 power를 표현하는 값을 log로 표현(dB로 표현)

# 시각화
plt.figure(figsize=(10, 4))
librosa.display.specshow(mel_db, sr=sr, hop_length=256, x_axis="time", y_axis="mel")
plt.title("녹음 음성의 Mel-Spectrogram")
plt.colorbar(format="%+2.0f dB")
plt.tight_layout()
plt.show()

In [ ]:
from g2pk import G2p

g2p = G2p()
#text를 사람이 발음하는 음소로 변환한다.
text = "안녕하세요, 오늘 날씨는 맑고 화창합니다."
phonemes = g2p(text)
phoneme_list = phonemes.split()

print("원문:", text)
print("음소:", phoneme_list)

In [ ]:
#R^256차원 내에 token을 embedding한다.
embedding_dim = 256
embedding_matrix = np.random.randn(len(phoneme_list), embedding_dim)

#y축은 음소, x축은 256차원 vector의 각 256개의 cell, 색은 vecotr의 각 cell에 저장된 값
plt.figure(figsize=(12, 3))
plt.imshow(embedding_matrix, aspect="auto", cmap="viridis")
plt.yticks(range(len(phoneme_list)), phoneme_list)
plt.colorbar()
plt.title("Phoneme Embedding 시각화")
plt.xlabel("임베딩 차원")
plt.show()

In [ ]:
#duration 설정
durations = [5] * len(phoneme_list)  # 예시용으로 5프레임씩 할당

plt.bar(phoneme_list, durations)
plt.title("Duration Predictor 출력")
plt.ylabel("예측된 프레임 수")
plt.show()

In [ ]:
expanded = np.concatenate([
    np.tile(embedding_matrix[i], (durations[i], 1))
    for i in range(len(phoneme_list))
], axis=0)

plt.figure(figsize=(12, 3))
plt.imshow(expanded, aspect="auto", cmap="plasma")
plt.title("Length Regulator 출력 (음소 길이 확장)")
plt.xlabel("임베딩 차원")
plt.ylabel("Time Frames")
plt.show()

In [ ]:
mel_output = np.random.randn(len(expanded), 80)

plt.figure(figsize=(10, 4))
plt.imshow(mel_output.T, aspect="auto", origin="lower", cmap="magma")
plt.title("FastSpeech Decoder 출력 (Mel-Spectrogram 예시)")
plt.xlabel("Time")
plt.ylabel("Mel bins")
plt.colorbar()
plt.show()

In [ ]:
# mel_output.shape = [T, 80]
# vocoder는 각 프레임(T)의 벡터(80차원)를 기반으로 짧은 오디오 조각을 예측하고, 그것들을 이어붙여 wave를 생성함

# T개의 프레임을 각각 파형 조각에 매핑한다고 가정하고 그 조각을 이어붙이는 과정 시뮬레이션
T = mel_output.shape[0]
waveform = np.concatenate([
    np.sin(2 * np.pi * np.linspace(0, 1, 200) * (i % 5 + 1)) * (mel_output[i].mean() * 0.1)
    for i in range(T)
])

# 정규화
waveform = waveform / np.max(np.abs(waveform))

# 시각화
plt.figure(figsize=(12, 2))
plt.plot(waveform, color="blue")
plt.title("Vocoder가 만든 가상의 음성 파형")
plt.xlabel("샘플 인덱스")
plt.ylabel("진폭")
plt.tight_layout()
plt.show()

HiFi-GAN Vocoder

입력: Mel-Spectrogram

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import librosa.display

T = 100  # time steps
mel_dim = 80
mel_input = np.random.randn(T, mel_dim)  # 예시 입력

plt.figure(figsize=(10, 3))
plt.imshow(mel_input.T, aspect="auto", origin="lower", cmap="magma")
plt.title("① 입력: Mel-Spectrogram")
plt.xlabel("Time Frames")
plt.ylabel("Mel Bins")
plt.colorbar(label="dB")
plt.tight_layout()
plt.show()

- Upsample: 시간 해상도 증가

HiFi-GAN은 ConvTranspose1D(Deconvolution)로 시간축을 늘려 wave에 맞춥니다.

In [ ]:
#sample의 시간 크기를 늘림
upsampled_time = T * 8  # 예시로 8배 업샘플링
upsampled = np.repeat(mel_input, 8, axis=0)

plt.figure(figsize=(10, 2))
plt.imshow(upsampled.T, aspect="auto", origin="lower", cmap="plasma")
plt.title("② Upsample: Mel 시간 해상도 증가")
plt.xlabel("Upsampled Time")
plt.ylabel("Mel Bins")
plt.tight_layout()
plt.show()

- MRF: 다양한 필터로 추출 (Multi-Receptive Field)

HiFi-GAN은 3가지 다른 커널 크기와 dilation을 가진 Conv 블록을 병렬 적용

In [ ]:
# MRF 구조 표현용 (세 개의 커널이 병렬로 특징 추출)
plt.figure(figsize=(12, 2))
for i, k in enumerate([3, 5, 7]):
    signal = np.convolve(upsampled[:, 0], np.ones(k)/k, mode='same')
    plt.plot(signal, label=f"커널 {k}", alpha=0.7)

plt.title("③ MRF: 다양한 수용 범위 필터 적용")
plt.legend()
plt.tight_layout()
plt.show()

Residual Blocks: 특징 깊이 있게 변환

In [ ]:
from scipy.ndimage import gaussian_filter1d

# Residual block 결과 시뮬레이션 (부드럽게 만듦)
residual_output = gaussian_filter1d(upsampled[:, 0], sigma=3)

plt.figure(figsize=(10, 2))
plt.plot(residual_output, label="Residual Block Output")
plt.title("④ Residual Block을 거친 특징 변환")
plt.tight_layout()
plt.show()

Output: 최종 Waveform 생성

In [ ]:
# 예시 파형 출력
waveform = residual_output
waveform = waveform / np.max(np.abs(waveform))  # 정규화

plt.figure(figsize=(12, 2))
plt.plot(waveform, color="blue")
plt.title("⑤ 최종 음성 Waveform (예시)")
plt.xlabel("샘플 인덱스")
plt.ylabel("진폭")
plt.tight_layout()
plt.show()